In [ ]:
# 02_disruption.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import ast
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
#直接加载01文件清洗后的数据
def load_cleaned_data(school='tsinghua_university'):
    """直接加载清洗后的CSV文件"""
    clean_file = Path('../data/cleaned') / f'{school}_cleaned.csv'
    
    if clean_file.exists():
        df = pd.read_csv(clean_file)
        print(f"✅ 加载清洗数据: {len(df)} 条记录")
        return df
    else:
        raise FileNotFoundError(f"清洗数据文件不存在: {clean_file}，请先运行01_data_cleaning.py")

# 加载数据
df = load_cleaned_data()
print(f"📊 数据形状: {df.shape}")
print(f"📅 年份范围: {df['year'].min()}-{df['year'].max()}")
print(f"📚 期刊数量: {df['source'].nunique()}")


✅ 加载清洗数据: 275 条记录
📊 数据形状: (275, 12)
📅 年份范围: 2010-2024
📚 期刊数量: 208


## 颠覆性

In [ ]:
class DisruptionIndexCalculator:   # 颠覆性指数计算类
    """计算论文颠覆性指数的类"""
    
    def __init__(self):
        self.citation_network = None  #用来存储"谁引用了谁"的关系
        self.paper_references = None  #用来存储每篇论文的参考文献列表
        
    def build_citation_network(self, df):
        """citing字段作为参考文献"""
        print("🔨 构建引文网络中...")
        
        # 初始化数据结构
        self.citation_network = defaultdict(set)  # {cited_paper: set(citing_papers)}
        self.paper_references = {}  # {paper_id: set(references)}
        
        # 处理引用关系
        valid_papers = 0
        for _, row in df.iterrows():
            paper_id = row['wos_id']
            
            # 处理参考文献 - 现在使用 citing 字段
            if 'citing' in df.columns and pd.notna(row.get('citing')):
                try:
                    # 解析citing字段作为参考文献列表
                    if isinstance(row['citing'], str):
                        # 尝试解析字符串格式的列表
                        citing_list = ast.literal_eval(row['citing'])
                    else:
                        citing_list = row['citing']
                    
                    # 保存当前论文的参考文献
                    self.paper_references[paper_id] = set(citing_list)
                    
                    # 构建引文网络：这些参考文献被当前论文引用
                    for ref in citing_list:
                        self.citation_network[ref].add(paper_id)
                    
                    valid_papers += 1
                    
                except (ValueError, SyntaxError) as e:
                    # 如果解析失败，设为空集合
                    self.paper_references[paper_id] = set()
                    print(f"⚠️ 解析citing字段失败: {paper_id}, 错误: {e}")
            else:
                # 没有citing数据，设为空集合
                self.paper_references[paper_id] = set()
        
        print(f"✅ 引文网络构建完成:")
        print(f"   - 总论文数: {len(df)}")
        print(f"   - 有效参考文献数据: {valid_papers}")
        print(f"   - 引文网络节点: {len(self.citation_network)}")
        
        return self
    
    def calculate_disruption_index(self, focal_paper_id):
        """计算单篇论文的颠覆性指数"""
        if self.citation_network is None or self.paper_references is None:
            raise ValueError("请先构建引文网络")
        
        # 获取焦点论文的参考文献（来自citing字段）
        references_R = self.paper_references.get(focal_paper_id, set())
        
        # 获取引用焦点论文的施引文献
        citing_C = self.citation_network.get(focal_paper_id, set())
        
        # 初始化计数器
        ni, nj, nk = 0, 0, 0
        
        # 统计 ni 和 nj
        for citing_paper in citing_C:
            # 获取施引文献的参考文献（同样来自citing字段）
            citing_refs = self.paper_references.get(citing_paper, set())
            
            # 判断引用模式
            cites_any_R = len(citing_refs & references_R) > 0
            
            if cites_any_R:
                nj += 1  # 既引FP又引R
            else:
                ni += 1  # 只引FP不引R
        
        # 统计 nk: 只引R不引FP的论文
        # 找到所有引用R但不引用FP的论文
        papers_citing_R = set()
        for ref_paper in references_R:
            papers_citing_R.update(self.citation_network.get(ref_paper, set()))
        
        # 排除那些也引用了FP的论文
        nk_candidates = papers_citing_R - citing_C
        nk = len(nk_candidates)
        
        # 计算D指数
        denominator = ni + nj + nk
        if denominator == 0:
            return 0.0, (ni, nj, nk)  # 避免除零错误
        
        d_index = (ni - nj) / denominator
        return d_index, (ni, nj, nk)
    
    def calculate_batch_disruption_indices(self, paper_ids):
        """批量计算颠覆性指数"""
        results = []
        
        print("📊 开始批量计算颠覆性指数...")
        total = len(paper_ids)
        
        for i, paper_id in enumerate(paper_ids):
            if i % 100 == 0:
                print(f"进度: {i}/{total} ({i/total*100:.1f}%)")
            
            try:
                d_index, counts = self.calculate_disruption_index(paper_id)
                results.append({
                    'wos_id': paper_id,
                    'disruption_index': d_index,
                })
            except Exception as e:
                print(f"❌ 计算失败 {paper_id}: {e}")
                # 添加错误记录
                results.append({
                    'wos_id': paper_id,
                    'disruption_index': np.nan,
                })
        
        return pd.DataFrame(results)


In [ ]:
# 计算颠覆性指数
def calculate_disruption_for_dataframe(df):
    """为整个DataFrame计算颠覆性指数"""
    
    # 初始化计算器
    calculator = DisruptionIndexCalculator()
    
    # 构建引文网络
    calculator.build_citation_network(df)
    
    # 获取所有论文ID
    paper_ids = df['wos_id'].tolist()
    
    # 批量计算颠覆性指数
    disruption_df = calculator.calculate_batch_disruption_indices(paper_ids)
    
    # 合并回原数据
    result_df = df.merge(disruption_df, on='wos_id', how='left')
    
    return result_df


# 执行计算
print("\n🎯 开始计算颠覆性指数...")

try:
    # 直接计算全部数据
    result_df = calculate_disruption_for_dataframe(df)
    
    # 显示结果统计
    print("\n📈 颠覆性指数统计:")
    disruption_stats = result_df['disruption_index'].describe()
    print(disruption_stats)
    
    # 显示分布
    print(f"\n📊 颠覆性指数分布:")
    print(f"  均值: {result_df['disruption_index'].mean():.4f}")
    print(f"  中位数: {result_df['disruption_index'].median():.4f}")
    print(f"  标准差: {result_df['disruption_index'].std():.4f}")
    print(f"  范围: [{result_df['disruption_index'].min():.4f}, {result_df['disruption_index'].max():.4f}]")
    
    # 保存结果
    output_file = Path('../data/analysis') / 'disruption_index_results.csv'
    output_file.parent.mkdir(exist_ok=True)
    result_df.to_csv(output_file, index=False)
    print(f"💾 结果已保存: {output_file}")
    
except Exception as e:
    print(f"❌ 计算失败: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# 期刊级别的颠覆性指标分析
def calculate_journal_disruption_metrics(result_df):
    """计算每个期刊的颠覆性指标"""
    
    print("\n📊 开始计算期刊级别颠覆性指标...")
    
    # 按期刊分组计算统计量
    journal_metrics = result_df.groupby('source').agg({
        'disruption_index': ['mean', 'median', 'std', 'count']
    }).round(4)
    
    # 扁平化列名
    journal_metrics.columns = ['disruption_mean', 'disruption_median', 'disruption_std', 'paper_count']
    
    # 重置索引
    journal_metrics = journal_metrics.reset_index()
    
    # 按颠覆性指数均值排序
    journal_metrics = journal_metrics.sort_values('disruption_mean', ascending=False)
    
    return journal_metrics

def visualize_journal_ranking(journal_metrics):
    """可视化期刊颠覆性排名"""
    
    plt.figure(figsize=(12, 8))
    
    # 期刊颠覆性排名条形图
    plt.barh(range(len(journal_metrics)), journal_metrics['disruption_mean'])
    plt.yticks(range(len(journal_metrics)), journal_metrics['source'], fontsize=8)
    plt.xlabel('平均颠覆性指数')
    plt.ylabel('期刊名称')
    plt.title('期刊颠覆性指数排名')
    plt.gca().invert_yaxis()  # 让最高的在最上面
    plt.grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.show()

# 执行期刊级别分析
try:
    # 计算期刊指标
    journal_metrics = calculate_journal_disruption_metrics(result_df)
    
    print(f"\n🎯 期刊颠覆性指标统计 (共{len(journal_metrics)}种期刊)")
    print("=" * 50)
    
    # 保存期刊指标
    journal_output_file = Path('../data/metrics') / 'journal_disruption_metrics.csv'
    journal_output_file.parent.mkdir(exist_ok=True)
    journal_metrics.to_csv(journal_output_file, index=False, encoding='utf-8-sig')
    print(f"💾 期刊指标已保存: {journal_output_file}")
    
    # 可视化
    visualize_journal_ranking(journal_metrics)

except Exception as e:
    print(f"❌ 期刊分析失败: {e}")
    import traceback
    traceback.print_exc()

,id,address,title,authors,pub_date,conference,source,citations,refs,wos_id,abstract,year
0,3,"Tsinghua Univ, Beijing, Peoples R China",The dual roles of lymphotoxin-β in promoting b...,"Wang, XX; Zheng, HQ",Sep 2024,NaN,NATURE CELL BIOLOGY,0,5,1302656500002,Using single-cell RNA sequencing analysis of b...,2024.0
1,5,"Tsinghua Univ, Beijing, Peoples R China",Predictive Effects of English Classroom Anxiet...,"Liu, MH; Li, TH",2024,NaN,JOURNAL OF LANGUAGE AND EDUCATION,0,68,1265026100006,Background: Second language (L2) learning is c...,2024.0
2,10,"Tsinghua Univ, Beijing, Peoples R China",Soju: A Global History,"Revells, T",Feb 2022,NaN,JOURNAL OF ASIAN STUDIES,0,1,768359300056,NaN,2022.0
3,11,"Tsinghua Univ, Beijing, Peoples R China",Lyricism and Implicit Emotional Expression of ...,"Chen, PH; Rau, PLP",2022,14th International Conference on Cross-Cultura...,CROSS-CULTURAL DESIGN-APPLICATIONS IN LEARNING...,0,47,1292829600012,Lots of music pieces are blooming with interne...,2022.0
4,12,"Tsinghua Univ, Beijing, Peoples R China",Statistical Algorithm Enables Rapid Computatio...,"Zhou, XY; Liu, XW; Ouyang, Z",Feb 2020,NaN,JOURNAL OF THE AMERICAN SOCIETY FOR MASS SPECT...,6,37,516075800030,Computation of the space charge effect within ...,2020.0


In [264]:
print(df.columns.tolist())


['id', 'address', 'title', 'authors', 'pub_date', 'conference', 'source', 'citations', 'refs', 'wos_id', 'abstract', 'year']
